# 10 Cross-Validation Techniques For You To Create Models That People Trust
## TODO
![](images/pexels.jpg)
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://www.pexels.com/@zen-chung?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Zen Chung</a>
        on 
        <a href='https://www.pexels.com/photo/crop-man-playing-with-dog-on-street-5749795/?utm_content=attributionCopyText&utm_medium=referral&utm_source=pexels'>Pexels.</a> All images are by the author unless specified otherwise.
    </strong>
</figcaption>

# Setup

In [1]:
import logging
import time
import warnings

import catboost as cb
import datatable as dt
import joblib
import lightgbm as lgbm
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import shap
import umap
import umap.plot
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.compose import *
from sklearn.impute import *
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import *

logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%d-%b-%y %H:%M:%S", level=logging.INFO
)
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings("ignore")
pd.set_option("float_format", "{:.5f}".format)

# Introduction

# Why cross-validate?

# 1. KFold

# 2. LeavePOut

# 3. ShuffleSPlit

# 4. StratifiedKFold

# 5. StratifiedShuffleSplit

# 6. TimeSeriesSplit

# Other CVs for non-IID data

# Summary